In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tweets = pd.read_csv('pro_china_full.csv')
tweets = tweets.dropna().reset_index() # some rows come in as blank so they need to be dropped - also need to reset index so they can match embeddings later

train, test = train_test_split(tweets, test_size=0.2, random_state = 0)

In [6]:
# this cell tests for invalid or blank text entries and prints their index if one comes up

for idx, tweet in enumerate(tweets['text']):
    try: 
        tokenizer(tweet, padding='max_length', max_length=130, return_tensors="pt")
    except:
        print(idx, tweet)

In [5]:
# reading in tensors from file

embeddings = torch.Tensor()

for i in range(39):
    filename = "embeddings/pro_china_embedding_" + str(i*1000) + ".pt"
    embeddings = torch.cat((embeddings, torch.load(filename)))
    
print(embeddings.shape, tweets.shape) # these should be in agreement

torch.Size([38646, 768]) (38646, 4)


In [6]:
X_train = embeddings[train.index].detach().numpy()
y_train = train['is_ccp']

print(X_train.shape, y_train.shape)

(30916, 768) (30916,)


In [12]:
# model = keras.Sequential()

# model.add(keras.layers.Dense(512, 
#                              activation = 'relu',
#                              kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
#                              bias_regularizer=keras.regularizers.l2(1e-4),
#                              activity_regularizer=keras.regularizers.l2(1e-5)))
# model.add(keras.layers.Dropout(0.1))
# model.add(keras.layers.Dense(1, activation = 'sigmoid'))

# # adam optimizer is a fancier version of gradient descent.  You can read more about it here: https://arxiv.org/pdf/1412.6980.pdf
# optimizer = keras.optimizers.Adam(clipvalue=1)

# model.compile(optimizer=optimizer,
#               loss='binary_crossentropy',  # From information theory notebooks.
#               metrics=['accuracy'])        # What metric to output as we train.

In [7]:
model = keras.Sequential()

#model.add(keras.layers.Normalization())

model.add(keras.layers.Dense(4, 
                             activation = 'relu'))

model.add(keras.layers.Dense(1, activation = 'sigmoid'))

#optimizer = keras.optimizers.Adam(clipvalue=1, learning_rate=0.0001)

model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy']) 

In [8]:
model.fit(X_train, y_train, epochs=25)

Epoch 1/25
967/967 [==============================] - 3s 1ms/step - loss: 0.4244 - accuracy: 0.8315
Epoch 2/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2969 - accuracy: 0.8794
Epoch 3/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2684 - accuracy: 0.8925
Epoch 4/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2508 - accuracy: 0.9016
Epoch 5/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2404 - accuracy: 0.9070
Epoch 6/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2325 - accuracy: 0.9095
Epoch 7/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2264 - accuracy: 0.9121
Epoch 8/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2215 - accuracy: 0.9131
Epoch 9/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2185 - accuracy: 0.9154
Epoch 10/25
967/967 [==============================] - 1s 1ms/step - loss: 0.2149 - accuracy: 0.9159

# Testing

In [9]:
X_test = embeddings[test.index].detach().numpy()
y_test = test['is_ccp']

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred_bool = np.where(y_pred >= 0.5, 1, 0).ravel() #DIY function to round outputs to 0 or 1

print(classification_report(y_test, y_pred_bool))
print(confusion_matrix(y_test, y_pred_bool))

              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93      3910
         1.0       0.93      0.92      0.93      3820

    accuracy                           0.93      7730
   macro avg       0.93      0.93      0.93      7730
weighted avg       0.93      0.93      0.93      7730

[[3661  249]
 [ 318 3502]]


In [11]:
model.evaluate(X_test, y_test)    

242/242 [==============================] - 0s 822us/step - loss: 0.1804 - accuracy: 0.9266


[0.1804370880126953, 0.9266493916511536]

In [16]:
# one line tester

line = "xinjiang is a slavery concentration camp"
line_token = tokenizer(line, padding='max_length', max_length=130, return_tensors="pt")
line_embed = bertweet(**line_token)

model.predict(line_embed.pooler_output.detach().numpy())

array([[0.9439023]], dtype=float32)

# Test on other set

In [ ]:
embeddings_all = torch.Tensor()

for i in range(39):
    filename = "embeddings/all_china_embedding_" + str(i*1000) + ".pt"
    embeddings = torch.cat((embeddings, torch.load(filename)))
    
print(embeddings.shape, tweets.shape)